# 第1章 Python数据模型
Python最好的品质之一是一致性。\
Python风格（Pythonic）的关键是它的设计思想，这种设计思想完全体现在Python的数据模型上，数据模型所提供的API为使用最地道的语言特性来构建你自己的对象提供了工具。\
数据模型其实是对Python框架的描述，规范了这门语言自身构建模块的接口。\

## 1.1 一摞Python风格的纸牌
本节利用一个简单例子来展示两个特殊方法`__getitem__`和`__len__`的实现。

In [2]:
import collections

# 首先用namedtuple构建一个简单的类表示一张纸牌
# namedtuple可以用来构建只有少数属性但没有方法的对象，有方法可以用class
Card = collections.namedtuple('Card', ['rank', 'suit'])

beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
# 一摞有序的纸牌
class FrenchDeck():
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    print(suits)
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

['spades', 'diamonds', 'clubs', 'hearts']


In [4]:
deck = FrenchDeck()
print(len(deck))

52


In [5]:
deck[0]

Card(rank='2', suit='spades')

In [6]:
deck[-1]

Card(rank='A', suit='hearts')

In [7]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [8]:
# 切片
deck[12::13] # 只看牌面是A的牌

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

In [9]:
from random import choice
# 从序列中随机选一个元素

choice(deck)

Card(rank='Q', suit='clubs')

In [10]:
# 迭代
# 两种方式都可以
# for card in deck._cards:
#     print(card)

for card in deck:
    print(card)
    break

Card(rank='2', suit='spades')


迭代通常是隐式的，如果一个集合类型没有实现`__contains__`方法，那么`in`运算符就会按顺序做一次迭代搜索。

In [11]:
Card('A', 'hearts') in deck

True

In [13]:
Card('A', 'beasts') in deck

False

In [14]:
FrenchDeck.ranks # a list

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

In [15]:
# 返回2的索引
print(FrenchDeck.ranks.index('2'))
print(FrenchDeck.ranks.index('A'))

0
12


In [16]:
# 排序
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

# 返回card对应的排序优先度
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank) # 因为存在JQKA，所以需要得到数字索引
    return rank_value * len(suit_values) + suit_values[card.suit] # 相同点数再根据花色判断

print(spades_high(Card('A', 'clubs')))
print(spades_high(Card('A', 'spades')))

48
51


In [17]:
for card in sorted(deck, key=spades_high):
    print(card)
#     break

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

通过实现`__len__`和`__getitem__`两个特殊方法，`FrenchDeck`可以像Python自有的序列数据类型一样，体现出Python语言的核心特性，比如迭代和切片，还可以使用标准库的一些操作，比如`random.choice`、`reversed`和`sorted`等函数。

In [20]:
# 按照目前的设计，FrenchDeck是不能洗牌的，因为这摞牌是immutable；除非破坏类的封装性，直接对列表_cards进行操作
# deck.sort() 不可行

deck._cards.sort() # 直接对列表进行操作可以，而且修改了deck的顺序

In [21]:
deck[0] # 验证已经修改了顺序

Card(rank='10', suit='clubs')

## 1.2 如何使用特殊方法
特殊方法是为了python解释器调用的，比如`__len__`，而我自己使用应该是`len(my_object)`。

如果是Python内置的类型，在调用`__len__`时，CPython其实返回的是`PyVarObject`里的`ob_size`属性，而非调用一个方法。`PyVarObject`是表示内存中长度可变的内置对象的C语言结构体。

有时候特殊方法会被隐式调用，比如`for i in x`其实用的是`iter(x)`，而这个函数背后用的是`x.__iter()__`方法。

直接调用特殊方法的概率应该低于我自己实现它的概率，除了`__init__`，目的是在你子类的`__init__`方法中调用超类的构造器。

通过内置函数(`len`，`iter`，`str`等)使用特殊方法是最好的选择。不要自己随意添加特殊方法。
### 1.2.1 模拟数值类型

本章实现一个二维向量类，可以使用`+`，`*`进行相加、标量乘法。遇到内置函数`abs`时返回向量的模。

In [1]:
print(abs(3))
print(abs(5.5))
# 如果是复数，返回复数的模

3
5.5


In [3]:
from math import hypot
help(hypot)

Help on built-in function hypot in module math:

hypot(x, y, /)
    Return the Euclidean distance, sqrt(x*x + y*y).



In [12]:
# 一个简单的二维向量类
class Vector():
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        # %r而非%d，这是一个好习惯，意味着Vector(1,2)和Vector('1','2')不一样
        return('Vector(%r, %r)' % (self.x, self.y)) 
        
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, vector):
        return(Vector(self.x+vector.x, self.y+vector.y))
    
    def __mul__(self, scalar):
        return(Vector(self.x*scalar, self.y*scalar))

In [14]:
v1 = Vector(2,3)
v1 # python会调用__repr__

Vector(2, 3)

In [18]:
v2 = Vector(3, 4)
v2

Vector(3, 4)

In [19]:
v1+v2

Vector(5, 7)

In [10]:
abs(v2)

5.0

In [11]:
v2*3

Vector(9, 12)

### 1.2.2 字符串表达形式
内置函数`repr`把一个对象以字符串形式表达出来以便辨认，即“字符串表达形式”，如果没有实现`__repr__`，可能打印的结果是`<Vecotr Object at xxxxx>`。

`%`格式化字符串是老的用法，`str.format`是新的用法，都利用了`repr`函数来获取对象的标准字符串表示形式。目前是两种用法并存的局面。

`__repr__`返回的字符串应该准确无歧义，尽可能表达该被打印的对象是如何创建的，本例采用类似*调用对象构造器*的表达形式。\
`__repr__`和`__str__`函数的区别是，后者在`str`函数或`print`使用的时候被调用，输出更为友好。但如果只实现一个，建议实现`__repr__`，因为如果没有`__str__`，而Python又需要使用时会用`__repr__`代替。反之不会。

[What is the difference between `__str__` and `__repr__`?](https://stackoverflow.com/questions/1436703/what-is-the-difference-between-str-and-repr/1436756#1436756)

### 1.2.3 算术运算符
`+`, `\`等中缀运算符的基本原则是不改变操作对象，而是产出一个新值。

### 1.2.4 自定义的布尔值
为了判断一个对象是真还是假，Python会调用`bool(x)`。一般情况下，我们自定义的类的实例被认为是真的，除非自己定义了`__bool__`或者`__len__`函数，如果没有`__bool__`，python会调用`__len__`，如果返回0，则`bool`会返回`False`。

## 1.3 特殊方法一览
增量赋值运算符是一种把中缀运算符变成赋值运算的捷径(`a = a * b`变成`a *= b`)。

## 1.4 为什么len不是普通方法？
实用胜于纯粹。（Python之禅）

如果`x`是一个内置类型的实例，CPython会直接从一个C结构体中读取对象的长度，完全不会调用任何方法，这是很快的，而这一操作很常见。`len`之所以不是普通方法，是为了让Python自带的数据结构可以走后门。

又由于`len`是特殊方法，我们也可以用于自定义数据类型，因此在保持内置类型的效率和保证语言一致性间找到了一个平衡点。

## 1.5 本章小结
1. 通过实现特殊方法，自定义数据类型可以表现得跟内置类型一样，从而写出更加pythonic的代码。
2. Python对象的一个基本要求是有合理的字符串表示形式，可以用`__repr__`和`__str__`来实现。
3. 对序列数据类型的模拟是特殊方法使用最多的地方。
4. Python通过运算符重载提供了丰富的数据类型，比如本章实现的Vector类。

元对象（metaobject）是指对建构语言本身来讲很重要的对象。元对象协议，协议可以看作接口，元对象协议是对象模型的同义词，它们的意思都是构建核心语言的API。一套丰富的元对象协议能让我们对语言进行拓展。
新概念：面向方面编程